<a href="https://colab.research.google.com/github/baiyinnamula/-Pandas-/blob/main/%E7%AC%AC10%E7%AB%A0Pandas%E6%95%B0%E6%8D%AE%E6%B8%85%E6%B4%97.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>